In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
from citipy import citipy
from datetime import datetime
import gmaps
import requests

# Import the API keys.
from config import weather_api_key
from config import g_key

In [4]:
vacation_df = pd.read_csv("Resources/vacation.csv")
vacation_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description,Hotel Name
0,1,Isla Mujeres,MX,2019-10-22 06:25:23,21.23,-86.73,82.40,83,5,9.17,0.0,0.0,clear sky,Hotel La Joya Isla Mujeres
1,8,Shingu,JP,2019-10-22 06:25:25,33.72,135.99,75.20,57,40,16.11,0.0,0.0,scattered clouds,新宮ユーアイホテル
2,12,Kapaa,US,2019-10-22 06:22:34,22.08,-159.32,80.60,83,90,14.99,0.0,0.0,overcast clouds,Courtyard by Marriott Kaua'i at Coconut Beach
3,19,Vaini,IN,2019-10-22 06:22:50,15.34,74.49,80.60,69,75,9.17,0.0,0.0,broken clouds,Dandeli Lake County
4,20,Ligayan,PH,2019-10-22 06:25:29,4.65,119.47,84.92,68,76,5.12,0.0,0.0,broken clouds,NaN


In [20]:
four_cities_df = vacation_df.loc[(vacation_df["City"] == "Vila Velha") | \
                                (vacation_df["City"] == "Itarema") | \
                                (vacation_df["City"] == "Morros") | \
                                 (vacation_df["City"] == "Carutapera")]
four_cities_df

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description,Hotel Name
24,89,Carutapera,BR,2019-10-22 06:25:50,-1.20,-46.02,79.92,82,34,12.62,0.0,0.0,scattered clouds,Lidera
39,139,Morros,BR,2019-10-22 06:26:04,-2.87,-44.04,78.80,88,20,9.17,0.0,0.0,few clouds,Pousada Quebra Anzol
122,472,Vila Velha,BR,2019-10-22 06:23:57,-3.71,-38.60,78.80,78,40,5.82,0.0,0.0,scattered clouds,Hotel Iparana Paradise
136,520,Itarema,BR,2019-10-22 06:23:52,-2.92,-39.92,77.22,73,73,7.16,0.0,0.0,broken clouds,Windvillage


In [5]:
Vila_Velha = vacation_df.loc[vacation_df["City"] == "Vila Velha"]
Vila_Velha

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description,Hotel Name
122,472,Vila Velha,BR,2019-10-22 06:23:57,-3.71,-38.6,78.8,78,40,5.82,0.0,0.0,scattered clouds,Hotel Iparana Paradise


In [6]:
Itarema = vacation_df.loc[vacation_df["City"] == "Itarema"]
Itarema

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description,Hotel Name
136,520,Itarema,BR,2019-10-22 06:23:52,-2.92,-39.92,77.22,73,73,7.16,0.0,0.0,broken clouds,Windvillage


In [7]:
Morros = vacation_df.loc[vacation_df["City"] == "Morros"]
Morros

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description,Hotel Name
39,139,Morros,BR,2019-10-22 06:26:04,-2.87,-44.04,78.8,88,20,9.17,0.0,0.0,few clouds,Pousada Quebra Anzol


In [8]:
Carutapera = vacation_df.loc[vacation_df["City"] == "Carutapera"]
Carutapera

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain in Last 3 Hours (inches),Snow in Last 3 Hours (inches),Weather Description,Hotel Name
24,89,Carutapera,BR,2019-10-22 06:25:50,-1.2,-46.02,79.92,82,34,12.62,0.0,0.0,scattered clouds,Lidera


In [42]:
Vila_Velha_loc = tuple(tuple(Vila_Velha[["Lat", "Lng"]].values)[0])
Vila_Velha_loc

(-3.71, -38.6)

In [43]:
Itarema_loc = tuple(tuple(Itarema[["Lat", "Lng"]].values)[0])
Itarema_loc

(-2.92, -39.92)

In [44]:
Morros_loc = tuple(tuple(Morros[["Lat", "Lng"]].values)[0])
Morros_loc

(-2.87, -44.04)

In [45]:
Carutapera_loc = tuple(tuple(Carutapera[["Lat", "Lng"]].values)[0])
Carutapera_loc

(-1.2, -46.02)

In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

In [46]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in four_cities_df.iterrows()]

In [50]:
waypoints = [Itarema_loc, Morros_loc]
waypoints

[(-2.92, -39.92), (-2.87, -44.04)]

In [63]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = four_cities_df[["Lat", "Lng"]]
max_temp = four_cities_df["Max Temp"]
fig = gmaps.figure(center=(0, -45.0), zoom_level=4.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, 
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
directions = gmaps.directions_layer(Vila_Velha_loc, Carutapera_loc, waypoints=waypoints, travel_mode='DRIVING')
fig.add_layer(directions)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))